In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# 1차 : 쿠팡 상품 리스트 추출
- 밀키트 : 502482
- 국탕전골 : 502483
- 덮밥/비빔밥 : 502484
- 스테이크/고기 502485
- 면/파스타/감바스 502486
- 분식 502487
- 어린이 만들기 겸용 502489
- 중식요리 502490
- 기타요리 502491

In [6]:
# 쿠팡 전체 html 추출 함수
def get_coupang_item(URL, user_agt):
    headers = {"User-Agent":user_agt,
           "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    res = requests.get(URL, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')
    return soup

In [17]:
cg_url = 'https://www.coupang.com/np/categories/'
categoryId = '502483'
page = '5'

URL = cg_url+categoryId+"?page="+page
user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

cg_soup = get_coupang_item(URL, user_agt)

In [24]:
cg_soup

<!DOCTYPE html>
<!--[if lte IE 7 ]><html class="lt-ie9 lt-ie8" lang="ko-KR"><![endif]--><!--[if IE 8 ]><html class="lt-ie9" lang="ko-KR"><![endif]--><!--[if (gte IE 9)|!(IE)]><!--><html lang="ko-KR"><!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="zaNrGtrOLMjglkziY2IvmL8dOXyCWHGArDHqFazJQVI" name="google-site-verification"/>
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="//cart.coupang.com" rel="dns-prefetch"/>
<link href="//assets.coupang.com" rel="dns-prefetch"/>
<link href="//assets2.coupang.com" rel="dns-prefetch"/>
<link href="//assets.coupangcdn.com" rel="dns-prefetch"/>
<link href="//asset1.coupangcdn.com" rel="dns-prefetch"/>
<link href="//private.coupang.com" rel="dns-prefetch"/>
<link href="//img1a.coupangcdn.com" rel="dns-prefetch"/>
<link href="//image1.coupangcdn.com" rel="dns-prefetch"/>
<link href="//thumbnail1.coupangcdn.com" rel="dns-prefetch"/>
<link href="//static.coupangcdn.com" rel="dns-prefetch"/>
<link href="//www.facebook.com"

# 1차: 필요 정보만 추출
- 상품id, 상품명, 정가, 할인율, 할인가, 별점, 리뷰수, 품절여부

In [51]:
cg_soup.select("#productList li")

[<li class="baby-product renew-badge is-oos" data-is-rocket="true" data-product-id="6716539725" data-ranking="" data-vendor-item-id="82828401272" id="6716539725">
 <a class="baby-product-link" data-is-ccid-eligible="false" data-is-rocket="true" data-item-id="15610263296" data-product-id="6716539725" data-sns-discount-rate="-1" data-vendor-item-id="82828401272" data-wow-only-instant-discount-rate="-1" href="/vp/products/6716539725?itemId=15610263296&amp;vendorItemId=82828401272&amp;sourceType=CATEGORY&amp;categoryId=502383">
 <dl class="baby-product-wrap" data-use-data="N">
 <dt class="image">
 
 </dt>
 <dd class="descriptions">
 <div class="badges">
 </div>
 <div class="name">
     심플리쿡 알마니 동태탕 (냉동), 1106g, 1개
 </div>
 <div class="price-area">
 <div class="price-wrap">
 <div class="price">
 <em class="sale">
 <strong class="price-value">26,620</strong>원
                     
                 
 
                 
                     <span class="badge rocket">
 <img alt="로켓배송" height="

In [80]:
#상품id, #data-item-id  #data-vendor-item-id
bady_pdt = cg_soup.select_one("a.baby-product-link")

productId = bady_pdt['data-product-id']
itemsId = bady_pdt['data-item-id']
vendorItemId = bady_pdt['data-vendor-item-id']

print(productId, itemsId, vendorItemId)

6716539725 15610263296 82828401272


In [69]:
# 상품명, 
name = cg_soup.select_one("div.name").text
name = name.replace('\n', '').strip().replace(", 1개", '')
name

'심플리쿡 알마니 동태탕 (냉동), 1106g'

In [70]:
try:
    # 정가 > 없으면 pass
    base_price = cg_soup.select_one("del.base-price").text
    base_price = base_price.replace("\n",'').strip()
    print(base_price)

    # 할인율 > 없으면 pass
    discount_pcg = cg_soup.select_one("span.discount-percentage").text
    print(discount_pcg)
except:
    pass

14,010
21%


In [71]:
# 할인가, 기본 가격
price_value = cg_soup.select_one("strong.price-value").text
price_value

'26,620'

In [57]:
#100g당 가격
unit_price = cg_soup.select_one("span.unit-price").text
unit_price = unit_price.replace("\n", '').strip()
unit_price

'(100g당 2,407원)'

In [72]:
try:
    # 별점,  > 없으면 pass
    star = cg_soup.select_one("span.star > em").text
    print(star)
    # 리뷰수,  > 없으면 pass
    rating_count = cg_soup.select_one("span.rating-total-count").text
    print(rating_count)
except:
    pass

4.5
(26)


In [59]:
try:
    # 품절여부 > 없으면 pass
    out_of_stock = cg_soup.select_one("div.out-of-stock").text
    out_of_stock = out_of_stock.replace("\n", '').strip()
    print(out_of_stock)
except:
    pass

'일시품절'

# 2차 : 구성 정보 추출
- 메뉴이름, 구성정보

In [82]:
product_url = 'https://www.coupang.com/vp/products/'
productId = '1866720935' #data-product-id
itemsId = '3172927841'  #data-item-id
vendorItemId = '71160443802' #data-vendor-item-id

URL = product_url+productId+'/items/'+itemsId+'/vendoritems/'+vendorItemId
pdt_soup = get_coupang_item(URL, user_agt)

In [89]:
all = pdt_soup.select('p')
for a in all:
    print(a.text)

{"rollbackInterstellar":false,"productId":1866720935,"itemId":3172927841,"vendorItemId":71160443802,"vendorItemCertifications":[],"returnPolicyVo":{"vendorItemReturnPolicyLayoutType":"RETAIL","vendorItemReturnNotice":{"returnCharge":"ㆍ와우멤버십 회원: 무료로 반품/교환 가능ㆍ와우멤버십 회원 아닌 경우:1) [총 주문금액] - [반품 상품금액] = 19,800원 미만인 경우 반품비 5,000원2) [총 주문금액] - [반품 상품금액] = 19,800원 이상인 경우 반품비 2,500원"},"vendorItemDeliveryNotice":{"supportSameDayFresh":true,"rocketInstall":false,"rocketFresh":true,"deliveryMethod":"신선/냉장/냉동","deliveryCompany":"","deliveryCharge":"무료배송- 로켓프레시 상품 중 15,000원 이상 구매 시 무료배송- 해당 서비스 가능 지역에 한함","batchShipment":"가능","shippingPeriod":null},"sellerDetailInfo":{"vendorName":null,"threePM":false,"threePC":false,"sellerWithRepPersonName":null,"retail":true,"repPhoneNum":null,"repPersonName":null,"repEmail":null,"repAddress2":null,"repAddress1":null,"repAddress":null,"ecommReportNum":null,"bizNum":null}},"productSellingPoints":[],"exemption":null,"essentials":[{"title":"제품명","description":"대한곱창 곱